<h1>Cross Encoder Scores</h1>
Pre-computer question - answer cross encoder scores for use in L2R model

See more <a href = "https://huggingface.co/cross-encoder/qnli-distilroberta-base">here</a>

In [2]:
import sys
sys.path.insert(1, '../code')

In [59]:
from utils import * 

import os 
import csv
import torch
import pandas as pd 

from tqdm import tqdm 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

<h2>Load documents</h2>

In [32]:
comments = load_docs("../data/ask_reddit_posts_augmented.jsonl", text_key = "comments")
posts = load_docs("../data/ask_reddit_posts_augmented.jsonl", text_key = "post")

<h2>Load model</h2>

In [33]:
model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/qnli-distilroberta-base')

In [34]:
tokenizer = AutoTokenizer.from_pretrained('cross-encoder/qnli-distilroberta-base')

In [132]:
docids = [doc[0] for doc in posts]

In [138]:
# do not rerun! this is about 5 hours
doc2text_score = {}
for i in tqdm(range(len(docids))): 
    docid = docids[i]
    post = posts[i][1]
    comment = comments[i][1]

    features = tokenizer(post, comment, padding = True, truncation = True, return_tensors = "pt")

    model.eval()
    with torch.no_grad():
        score = torch.nn.functional.sigmoid(model(**features).logits)
        doc2text_score[docid] = float(score[0][0])

    

100%|███████████████████████████████████████| 200001/200001 [4:49:04<00:00, 11.53it/s]


In [141]:
with open("../models/cross_encoder_scores.csv", "a") as f:
    writer = csv.writer(f)
    for k, v in list(doc2text_score.items()):
        writer.writerow([k, v])